In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib qt

bootstrap = pd.read_csv('bootstrap_results_mitosisFeats_Distant Metastasis_Endocrine_LN0_stdThresh0_NEWcorrThresh1.csv')
# identify top feature from all experiments
feat_cols = bootstrap.columns[1:-2]

# converting bootsrap features to log space
bootstrap[feat_cols] = np.log(bootstrap[feat_cols])

print('c-index', bootstrap['c_index'].mean())
print('p-value', 2*bootstrap['p_value'].median())


c-index 0.7167726460311263
p-value 0.0212197461107722


inotify_add_watch(/home/mostafa/.config/ibus/bus/3a0072272c814e94aa7a81e7d532728a-unix-0) failed: (No space left on device)


In [2]:
hrs_mean = bootstrap[feat_cols].mean(axis=0).abs()

# finding the top features based on stats
num_top_features = 3
temp = hrs_mean.sort_values(ascending=False)
top_features = temp.index.to_list()[:num_top_features]
# top_features = ['M', 'LVI']

# top features to dataframe
top_hazard_ratios = bootstrap[top_features]

In [3]:
plt.plot(bootstrap['c_index'])

In [3]:
def set_axis_style(ax, labels, pos):
    ax.yaxis.set_tick_params(direction='out')
    ax.yaxis.set_ticks_position('left')
    ax.set_yticks(pos, labels=labels)
    ax.set_ylim(0.25, len(labels) + 0.75)
    # ax.set_xlabel('Sample name')

In [4]:
## combine these different collections into a list
# data_to_plot = [np.log(top_hazard_ratios[feat].to_list()) for feat in top_features]
plt.figure()
data_to_plot = [top_hazard_ratios[feat].to_list() for feat in top_features]
pos = len(top_features) - np.arange(len(top_features))
plt.violinplot(data_to_plot, pos,showmeans=True, showextrema=True, showmedians=True,vert=False)
ax = plt.gca()
set_axis_style(ax, top_features, pos)
plt.show()

In [5]:
#### Now, we first identify experiments with high c-index and isolate them
ex_ids = np.where(bootstrap['c_index'].to_numpy()>0.65)[0]
selected_bootsrap = bootstrap.iloc[ex_ids.tolist()]

# identify top feature from all experiments
feat_cols = selected_bootsrap.columns[1:-2]
hrs_mean = selected_bootsrap[feat_cols].mean(axis=0).abs()

# finding the top features based on stats
num_top_features = 50
temp = hrs_mean.sort_values(ascending=False)
top_features = temp.index.to_list()[:num_top_features]

# top features to dataframe
top_hazard_ratios = selected_bootsrap[top_features]

In [6]:
plt.figure(),
# data_to_plot = [np.log(top_hazard_ratios[feat].to_list()) for feat in top_features]
data_to_plot = [top_hazard_ratios[feat].to_list() for feat in top_features]
pos = len(top_features) - np.arange(len(top_features))
plt.violinplot(data_to_plot, pos,showmeans=True, showextrema=True, showmedians=True,vert=False)
ax = plt.gca()
set_axis_style(ax, top_features, pos)
plt.show()

In [16]:
import pandas as pd
discov_df = pd.read_csv('all_feats_combi/features_combined/NOTT/discovery_valid_combined.csv')
all_feats_list = discov_df.columns[39:]
std_thresh = 0.1
corr_thresh = 0.7

features_std = discov_df.std()
feats_list = []
for feat in all_feats_list:
    if features_std[feat] > std_thresh:
    #    print(feat, features_std[feat])
        feats_list.append(feat)
print(len(feats_list))     

normalized_df = (discov_df[feats_list] - discov_df[feats_list].min()) / (discov_df[feats_list].max() - discov_df[feats_list].min())
corr = normalized_df[feats_list].corr().abs()
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool))
corrolated_features = [column for column in upper_tri.columns if any(upper_tri[column] > corr_thresh)]
final_feat_list = list(set(feats_list).difference(set(corrolated_features)))
print(len(final_feat_list))

/tmp/ipykernel_15446/522006459.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  discov_df = pd.read_csv('all_feats_combi/features_combined/NOTT/discovery_valid_combined.csv')
/tmp/ipykernel_15446/522006459.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  features_std = discov_df.std()


525
116


In [3]:

bootsrap_num = 5
discov_val_feats_path = 'all_feats_combi/features_combined/NOTT/discovery_valid_combined.csv'
event_col = 'Distant Metastasis'
discov_df = pd.read_csv(discov_val_feats_path)
EE = discov_df[event_col].to_numpy()
rng = np.random.RandomState()
c_indices = []
p_values = []

train_list = []
for run in range(bootsrap_num):
    pos_index_train = list(rng.choice(np.nonzero(EE==1)[0],size = np.sum(EE)))
    train_list.append(pos_index_train)
    

In [12]:
np.where(discov_df.columns=='nodeDegrees_std')[0]

array([309])

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df = discov_df[['assortCoeff', 'avrDegree_1', 'avrDegree_10', 'avrDegree_2',
       'avrDegree_5', 'avrDegree_7', 'cenCloseness_h4', 'cenCloseness_std',
       'cenDegree_h4', 'cenDegree_h5', 'cenDegree_max', 'cenEigen_h3',
       'cenEigen_h5', 'cenEigen_max', 'cenEigen_med', 'cenKatz_h2',
       'cenKatz_h3', 'cenKatz_h5', 'mit_tum_ratio', 'nodeDegrees_h1']]

X = df.assign(const=1)
mosi = pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

In [16]:
feature_scores = {
    'c_index': [0.7, 0.7, 0.8, 0.9, 0.6, 0.5, 0.65, 0.7],
    'p_value': [0.05, 0.1, 0.4, 0.01, 0.002, 0.3, 0.06, 0.1],
    'num_fail':[1, 3, 2, 0, 3, 5, 0, 2]}

score_df = pd.DataFrame(feature_scores)
print(score_df)
sorted_score_df = score_df.sort_values(['c_index', 'p_value', 'num_fail'], ascending=[False, True, True])
print(sorted_score_df.index[0])
print(f'Best feature scores: {sorted_score_df.iloc[0][0]}')


   c_index  p_value  num_fail
0     0.70    0.050         1
1     0.70    0.100         3
2     0.80    0.400         2
3     0.90    0.010         0
4     0.60    0.002         3
5     0.50    0.300         5
6     0.65    0.060         0
7     0.70    0.100         2
3
Best feature scores: 0.0


In [7]:
list(bootstrap.columns)



['Unnamed: 0',
 'cenHarmonic_max',
 'nodeDegrees_max',
 'clusterCoff_med',
 'c_index',
 'p_value']